---
**Author:** Arifa Kokab  
**For:** AAI-540 Machine Learning Operations  
**Institution:** University of San Diego

# Model Creation & Registration for Facial Expression Analysis (MobileNetV2)

## Introduction
This notebook covers the final stage of model creation for a facial expression analysis project using MobileNetV2.  
The model was initially trained on Google Colab using a high-performance NVIDIA A100 GPU to accelerate deep learning experiments and reduce training time.  
After training, the model artifact was exported and imported into AWS SageMaker for packaging, registration, and deployment. 

## Model Training Provenance

- **Training platform:** Google Colab with NVIDIA A100 GPU
- **Reason:** Faster deep learning training
- **Exported model:** Saved as `.tar.gz` and uploaded to S3 for AWS SageMaker registration and deployment

This approach demonstrates flexibility in utilizing the best available resources for each stage of the MLOps pipeline.

---

## 1. Setup: Import Libraries and Initialize SageMaker Session

This section imports all required libraries and initializes your AWS SageMaker session, execution role, and Boto3 client to interact with SageMaker resources programmatically.

In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
import sagemaker
import boto3

role = get_execution_role()
sess = sagemaker.Session()

## 2. Model Packaging: Define and Configure PyTorch Model

Here, we define the `PyTorchModel` object with the required inference script, model artifact location, framework and Python versions, and session/role parameters. This prepares the trained MobileNetV2 model for registration and deployment.

In [1]:
model = PyTorchModel(
    entry_point='inference.py',
    model_data='s3://sagemaker-us-east-1-301806113644/feamodelfolder/mob_V2model.tar.gz',
    role=role,
    framework_version='1.9.1',
    py_version='py38',
    sagemaker_session=sess,
    name="FacialExpressionAnalysisFinal"
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
✓ Model registered in SageMaker as 'FacialExpressionAnalysisFinal'


## 3. Model Registration: Create SageMaker Model from Artifact

We now register the packaged model with SageMaker, creating a deployable model entity in your account. This step makes the trained model available for deployment, batch inference, or further automation in SageMaker pipelines.

In [ ]:
sagemaker_model = model.create(instance_type='ml.m5.large')
print("✓ Model registered in SageMaker as 'FacialExpressionAnalysisFinal'")

## 4. Model Verification: Retrieve and Display Model Details

To confirm successful registration, we query SageMaker for the model’s details—such as the Model ARN, container image, S3 location, and execution role. This ensures traceability and governance for downstream deployment and audit.

In [1]:
model_name = "FacialExpressionAnalysisFinal"
sm_client = boto3.client('sagemaker')

try:
    response = sm_client.describe_model(ModelName=model_name)
    print(f"Model '{model_name}' is registered in SageMaker.")
    print("----- Model Details -----")
    print(f"Model ARN: {response['ModelArn']}")
    print(f"Primary container image: {response['PrimaryContainer']['Image']}")
    print(f"Model Data URL: {response['PrimaryContainer']['ModelDataUrl']}")
    print(f"Execution Role ARN: {response['ExecutionRoleArn']}")
    print(f"Creation Time: {response['CreationTime']}")
except sm_client.exceptions.ClientError as e:
    print(f"Model '{model_name}' not found or not active.")
    print(str(e))

Model 'FacialExpressionAnalysisFinal' is registered in SageMaker.
----- Model Details -----
Model ARN: arn:aws:sagemaker:us-east-1:301806113644:model/FacialExpressionAnalysisFinal
Primary container image: 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.9.1-cpu-py38
Model Data URL: s3://sagemaker-us-east-1-301806113644/FacialExpressionAnalysisFinal/model.tar.gz
Execution Role ARN: arn:aws:iam::301806113644:role/LabRole
Creation Time: 2025-06-24 04:59:30.227000+00:00


## Conclusion

This notebook demonstrates the packaging and registration of a production-ready MobileNetV2 facial expression analysis model in AWS SageMaker. The resulting model can now be deployed, versioned, and integrated into automated CI/CD workflows for real-world neuromarketing applications.